In [1]:
import pandas as pd
from sklearn import datasets
import numpy as np

#Taking it in as an np array
df = datasets.load_iris()

#converting it to a pandas dataframe
df = pd.DataFrame(data=np.c_[df['data'], df['target']],
                  columns= df['feature_names'] + ['target']) 

df


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [2]:
df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [3]:
df.target.value_counts()

0.0    50
1.0    50
2.0    50
Name: target, dtype: int64

In [19]:
#defining X and y
X = df.drop(['target'], axis=1)
y = df['target']


In [23]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from numpy import mean
from joblib import dump

model  = KNeighborsClassifier(n_neighbors=3)

def cross_val_eval(model, X, y):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)
    cv_scores = cross_val_score(model, X, y, cv = cv, scoring='accuracy', n_jobs=-1, error_score='raise')
    return cv_scores

cv_scores = cross_val_eval(model, X, y)
print(mean(cv_scores))

model.fit(X, y)
dump(model, 'model.joblib')





0.9533333333333334


['model.joblib']

In [25]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from numpy import std
from numpy import mean
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import StackingClassifier



def get_models():
    models = dict()
    models['knc'] = KNeighborsClassifier()
    models['dtc'] = DecisionTreeClassifier()
    models['xgc'] = XGBClassifier()
    models['mlp'] = MLPClassifier(max_iter=1000, random_state = 0)   
    return models

def cross_val_eval(model, X, y):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)
    cv_scores = cross_val_score(model, X, y, cv = cv, scoring='accuracy', n_jobs=-1, error_score='raise')
    return cv_scores

models = get_models()

#model evaluation
results, names = list(), list()
for name, model in models.items():
    scores = cross_val_eval(model, X, y)
    results.append(scores)
    names.append(names)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))


>knc 0.947 (0.045)
>dtc 0.940 (0.039)
>xgc 0.947 (0.016)
>mlp 0.967 (0.030)


In [26]:
#stacking model
def model_stack():
    level0 = list()
    level0.append(('knc', KNeighborsClassifier()))
    level0.append(('dtc', DecisionTreeClassifier()))
    level0.append(('xgc', XGBClassifier()))
    #level1 
    level1 = MLPClassifier(max_iter=1000, random_state = 0)
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
    return model

def get_models():
    models = dict()
    models['knc'] = KNeighborsClassifier()
    models['dtc'] = DecisionTreeClassifier()
    models['xgc'] = XGBClassifier()
    models['mlp'] = MLPClassifier(max_iter=1000, random_state = 0)   
    models['StackingClassifier'] = model_stack()
    return models

models = get_models()

results, names = list(), list()
for name, model in models.items():
    scores = cross_val_eval(model, X, y)
    results.append(scores)
    names.append(names)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))


>knc 0.947 (0.045)
>dtc 0.953 (0.016)
>xgc 0.947 (0.016)
>mlp 0.967 (0.030)


/home/mwaniki-new/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/mwaniki-new/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/mwaniki-new/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/mwaniki-new/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


>StackingClassifier 0.953 (0.034)


/home/mwaniki-new/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
